(solph-linear-tespy)=

# Combining oemof-solph and TESPy

After running the TESPy model to determine the COP at different ambient temperature values, the next steps are as
follows:

- read the results fromt he TESPy model,
- make an interpolation to map the results to the actual ambient temperature time series and
- pass the COP time series to the heat pump in the `oemof-solph` model.

```{note}
The `oemof-solph` model will have the identical components as in the previous model. The only change is, that we use a
different source for our COP time series. We will therefore import a ready to use `oemof-solph` `EnergySysten` from our
utilities script to save some space here and only add the heat pump.
```

In [ ]:
from utilities import load_input_data, create_energy_system_stub
input_data = load_input_data().head(24*3)

es, bus_electricity, bus_heat_35C, thermal_storage, electricity_grid = create_energy_system_stub(input_data)

## Build the heat pump model

To build the heat pump model we load the COP from the TESPy simulation first and then map the ambient temperature data
from our ambient temperature time series to the TESPy lookup data.

In [ ]:
from utilities import load_tespy_cop

tespy_cop = load_tespy_cop()
tespy_cop

We can compare the TESPy COP with the datasheet and constant efficiency factor approach in
{numref}`cop-timeseries-tespy-vs-efficiency-factor`. They are, as expected, very similar.

```{glue:figure} cop-timeseries-tespy-vs-efficiency-factor
:name: "cop-timeseries-tespy-vs-efficiency-factor"

Comparison of the COP time series from the constant efficiency method with the TESPy model.
```

In [ ]:
input_data["simple TESPy COP"] = input_data["Ambient temperature (d°C)"].map(tespy_cop["COP"])


datasheet_cop = 4.9
carnot_cop_2_40 = (40+273.15) / (40-2)
cpf_2_40 = datasheet_cop / carnot_cop_2_40

input_data["cpf COP 2 -> 40"] = cpf_2_40 * (40 + 273.15) / (40 - input_data["Ambient temperature (°C)"] + 5)


from matplotlib import pyplot as plt


fig, ax = plt.subplots(1)

ax.plot(input_data["Ambient temperature (d°C)"]/10, "b-")
ax.set_ylabel("Ambient temperature (°C)").set_color("blue")
ax.set_xlabel("timestamp")

ax2 = ax.twinx()

ax2.plot(input_data["simple TESPy COP"], "r-")
ax2.plot(input_data["cpf COP 2 -> 40"], "r--")
ax2.set_ylabel("COP").set_color("red")

plt.close()


In [ ]:
from myst_nb import glue
glue("cop-timeseries-tespy-vs-efficiency-factor", fig, display=False)

In [ ]:
import oemof.solph as solph


hp_thermal_power = 9.1  # kW
cop = input_data["simple TESPy COP"][:-1]

heat_pump = solph.components.Transformer(
    label="heat pump",
    inputs={bus_electricity: solph.Flow()},
    outputs={bus_heat_35C: solph.Flow(nominal_value=hp_thermal_power)},
    conversion_factors={
        bus_electricity: 1 / cop,
        bus_heat_35C: 1,
    },
)

es.add(heat_pump)

## Run the model

Finally we can run our optimization model and read the results again.

In [ ]:
model = solph.Model(energysystem=es)
model.solve()
results = solph.processing.results(model)

In [ ]:
import numpy as np

demand = input_data["Heat load (kW)"][:-1]


heat_supply = results[(heat_pump, bus_heat_35C)]["sequences"]["flow"]
storage_content = results[(thermal_storage, None)]["sequences"]["storage_content"]

plt.plot(np.sort(heat_supply)[::-1], "r-", label="heat supply")
plt.plot(np.sort(demand)[::-1], "b--", label="heat demand")
plt.ylabel("Power (kW)")
plt.grid()
plt.legend()

plt.figure()
plt.plot(heat_supply, "r-", label="heat supply", drawstyle="steps-post")
plt.plot(demand, "b--", label="heat demand", drawstyle="steps-post")
plt.plot(storage_content, "k-", label="storage content")
plt.ylabel("Power (kW) or Energy (kWh)")
plt.grid()
plt.legend()

electricity_consumption = float(results[(electricity_grid, bus_electricity)]["sequences"].sum())
print(f"Electricity demand: {electricity_consumption:.1f} kWh")